In [174]:
import pandas as pd
import json
import numpy as np
import pymongo
from pymongo import MongoClient
import difflib

In [2]:
client = MongoClient('127.0.0.1', 27017)
db = client.aoligei
imdb_collection = db.imdb_movies
tmdb_collection = db.tmdb_movies
reviews = db.reviews

In [73]:
tmdb_keys = ['genres','vote_average','vote_count','poster_path','original_language','popularity','budget'
             ,'revenue','status','tagline','homepage','belongs_to_collection','release_date'
             ,'overview','originalTitle','runtimeMinutes','company_name','keywords_name','num_of_cast'
             ,'top_10_cast_popularity_mean','casts','top_10_cast_popularity','num_of_crew','Writers'
             ,'top_10_crew_popularity_mean','crews','top_10_crew_popularity','Directors']

In [94]:
#remove null valued key-value pairs to save space
for key in tmdb_keys:
    tmdb_collection.update_many(
#         {key: None},
        { '$or': [ { key: None }, { key: '' }, { key: [] } ] },
        {'$unset': { key: ''}}   
    )

In [121]:
tmdb_collection.update_many( {}, { '$rename': { "vote_average": "tmdb_avgRating" } } )

In [123]:
tmdb_collection.update_many( {}, { '$rename': { "vote_count": "tmdb_numVotes" } } )

In [125]:
itry = db.itry
ttry = db.ttry

In [126]:
itry.insert_many(imdb_collection.find()[0:20])

In [128]:
ttry.insert_many(tmdb_collection.find()[0:20])

In [192]:
def entity_res(list1, list2):
    for i in list2:
        if len(difflib.get_close_matches(i, list1, cutoff=0.7)) == 0:
               list1.append(i)
    return list1

In [251]:
#project out 'release_date': 0, 'primaryTitle': 0, 'originalTitle': 0, 'isAdult': 0
tmdb_iterator = tmdb_collection.find({}, {'release_date': 0, 'primaryTitle': 0, 'originalTitle': 0, 'isAdult': 0})

In [252]:
for tmovie in tmdb_iterator:
    if imdb_collection.find({'_id': _id}).count() == 0:
        break
    _id = tmovie['_id']
    imovie = imdb_collection.find({'_id': _id})[0]
    for key in tmovie.keys():
        #always use the casts from tmdb for popularity feature that only tmdb has
        if key == 'casts':
            imdb_collection.update_one({'_id': _id}
                            , {'$set': {key: tmovie[key]}})
        elif key == 'Directors' and key in imovie:
            imdb_collection.update_one({'_id': _id}
                            , {'$set': { key: entity_res(imovie[key], [tmovie[key]]) }})
        elif key == 'genres' and key in imovie:
            imdb_collection.update_one({'_id': _id}
                            , {'$set': { key: entity_res(imovie[key], tmovie[key]) }})  
        elif key == 'Writers' and key in imovie:
            imdb_collection.update_one({'_id': _id}
                            , {'$set': { key: entity_res(imovie[key], tmovie[key]) }})  
        else:
            imdb_collection.update_one( 
                {'_id': _id, key:{ '$exists': 0 }}
                , {'$set': {key: tmovie[key]}} )
    

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  


KeyboardInterrupt: 

In [169]:
imdb_collection.find({'_id':'tt0000574'})[0]

{'_id': 'tt0000574',
 'primaryTitle': 'The Story of the Kelly Gang',
 'originalTitle': 'The Story of the Kelly Gang',
 'isAdult': 0,
 'release_year': 1906,
 'runtimeMinutes': 70.0,
 'genres': ['Action', 'Adventure', 'Biography'],
 'imdb_avgRating': 6.1,
 'imdb_numVotes': 622.0,
 'Directors': ['Charles Tait'],
 'casts': ['Elizabeth Tait', 'John Tait', 'Norman Campbell', 'Bella Cola']}

In [170]:
tmdb_collection.find({'_id':'tt0000574'})[0]

{'_id': 'tt0000574',
 'tmdb_id': 20105,
 'genres': ['History', 'Action', 'Drama', 'Crime'],
 'poster_path': 'https://www.themoviedb.org/t/p/w600_and_h900_bestv2/s04HLqRn2j6wftUYqFDMxtZP3uE.jpg',
 'original_language': 'en',
 'popularity': 2.764,
 'budget': 0,
 'revenue': 0,
 'status': 'Released',
 'tagline': 'The Most Sensational, The Most Thrilling and Interesting LIVING PICTURES EVER TAKEN.',
 'release_date': '1906-12-26',
 'overview': 'Just as Fritz Lang’s Metropolis (1927) is testimony to German silent film art, The Story of the Kelly Gang (1906) symbolises both the birth of the Australian film industry and the emergence of an Australian identity. Even more significantly it heralds the emergence of the feature film format.  The Story of the Kelly Gang, directed by Charles Tait in 1906, is the first full-length narrative feature film produced anywhere in the world.  Only fragments of the original production of more than one hour are known to exist and are preserved at the National Fi